In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.lines import Line2D
from math import radians, sin, cos, asin, sqrt
import re
import requests
from datetime import time, timedelta, datetime
from statistics import median, quantiles


import geohash2
import pyproj
from pyproj import Proj, transform, CRS
from functools import partial
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
from scipy.spatial import cKDTree
from scipy.stats import poisson

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ortools.linear_solver import pywraplp

import random
import pickle
from tqdm import tqdm
from time import sleep

# Data Loading

## Loading functions

In [ ]:
def calculate_demand_data(row):
    probabilities = {
        '0-10': 0,
        '10-20': 0.003458068783068975,
        '20-30': 0.013896825396825975,
        '30-40': 0.02136825396825512,
        '40-50': 0.02338333333333506,
        '50-60': 0.026277248677250214,
        '60-70': 0.03332089947090043,
        '70-80': 0.046515343915346036,
        '80+': 0.046515343915346036,
    }

    total_demand = 0
    for age_group, count in row['Alter'].items():
        lambda_value = count * probabilities[age_group]
        total_demand += poisson.rvs(lambda_value)
    return total_demand


def setup_customer_data(folder, city):
    customers_gdf = gpd.read_file(f'./{folder}/cluster_{city}.gpkg')
    customers_gdf['Alter'] = customers_gdf['Alter'].apply(json.loads)
    # customers_gdf['nachfrage'] = 0
    # for i in range(365):
    #    customers_gdf['nachfrage'] += customers_gdf.apply(calculate_demand_data, axis=1)

    # bevoelkerung_sum = customers_gdf[customers_gdf['cluster'] != -1].groupby('cluster')['sum_INSGESAMT_0'].sum()
    # noise_demand = customers_gdf[customers_gdf['cluster'] == -1]['nachfrage'].iloc[0]

    # # Verteilung des Noise-Bedarfs auf die anderen Cluster anteilig zur Anzahl der Polygone
    # for cluster in bevoelkerung_sum.index:
    #     cluster_demand = noise_demand * (bevoelkerung_sum[cluster] / bevoelkerung_sum.sum())
    #     customers_gdf.loc[customers_gdf['cluster'] == cluster, 'nachfrage'] += np.round(cluster_demand)

    # customers_gdf = customers_gdf[customers_gdf['cluster'] != -1].reset_index(drop=True)
    # # Set Index to cluster id
    # customers_gdf.set_index(['cluster'], inplace=True)

    return customers_gdf


def load_energy_costs():
    # Calculate timestamps (current time minus 48 hours and current time)
    current_time = datetime.now()
    past_timestamp = (current_time - timedelta(days=365)).timestamp() * 1000
    current_timestamp = current_time.timestamp() * 1000

    # Construct the API URL with updated timestamps
    api_url = f"https://api.awattar.de/v1/marketdata?start={past_timestamp}&end={current_timestamp}"

    # Send GET request using the requests library
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for non-2xx status codes
    except requests.exceptions.RequestException as error:
        print(f"Error: {error}")

    # Load data in json format
    data_energy = json.loads(response.content)
    # Extract market prices
    market_prices = [item["marketprice"] for item in data_energy["data"]]

    # Calculate quantiles
    q1, q2, q3 = quantiles(market_prices)  # Use quartiles function
    # Transform to Eur/kWh
    q1_kwh_eur = q1 / 1000
    q2_kwh_eur = q2 / 1000
    q3_kwh_eur = q3 / 1000

    print(f"Median market price (kWh): {q2_kwh_eur:.5f} Eur/kWh")
    print(f"25th percentile (Q1, kWh): {q1_kwh_eur:.2f} Eur/kWh")
    print(f"75th percentile (Q3, kWh): {q3_kwh_eur:.2f} Eur/kWh")

    return data_energy, q1_kwh_eur, q2_kwh_eur, q3_kwh_eur


def calculate_travel_distance(warehouses_gdf, customers_gdf):
    # Create an empty list to store the results
    data = []
    # Iterate through each warehouse in the warehouse DataFrame
    for warehouse_index, warehouse in warehouses_gdf.iterrows():
        for customer_index, customer in customers_gdf.iterrows():
            # Calculate the distance between the centroid of the region and the warehouse
            travel_distance = warehouse.geometry.distance(customer.geometry.centroid)/1000
            # Append the calculated values to the list
            data.append({'warehouse_id': warehouse_index, 'region_id': customer_index, 'travel_distance': travel_distance})
    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    return df


def load_data(customers_gdf, folder, city):
    
    # Grenzen des Simulationsrahmens laden
    geo_würzburg = gpd.read_file(f'./{folder}/geo_{city}.gpkg')

    # Detaillierte Gebäude/Personen Daten laden
    bevölkerungs_gdf = gpd.read_file(f'./{folder}/pharmacy_assigned_complete.gpkg')
    bevölkerungs_gdf['Alter'] = bevölkerungs_gdf['Alter'].apply(json.loads)
    bevölkerungs_gdf['Geschlecht'] = bevölkerungs_gdf['Geschlecht'].apply(json.loads)

    # Apotheken Daten laden
    pharmacy_df = pd.read_csv(f'./{folder}/{city}-Apotheken.csv')
    pharmacy_gdf = gpd.GeoDataFrame(pharmacy_df, geometry=gpd.points_from_xy(pharmacy_df['lon'], pharmacy_df['lat']), crs=CRS("EPSG:4326"))
    pharmacy_gdf = pharmacy_gdf.to_crs(bevölkerungs_gdf.crs)

    # Warehouse Daten laden
    warehouses_gdf = gpd.read_file(f'./{folder}/warehouses_{city}.gpkg')

    # Distanzmatrix der Cluster und Warehouses erstellen
    shifts_df = calculate_travel_distance(warehouses_gdf, customers_gdf)
    shifts_df.set_index(['warehouse_id', 'region_id'], inplace=True)

    return geo_würzburg, bevölkerungs_gdf, pharmacy_gdf, warehouses_gdf, shifts_df


# Plotting the Data

In [ ]:
# Rahmen der Grafik definieren
fig, ax = plt.subplots(figsize=(6, 6))
geo_würzburg.boundary.plot(ax=ax, color='gray', linewidth=0.5)

# Plotte gdf_loaded auf dieselbe Achse
pharmacy_gdf.plot(ax=ax, markersize=20)

# Titel hinzufügen
plt.title('Apothekenstandorte im Landkreis Würzburg')
plt.axis('off')

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Zeige den Plot an
plt.show()

In [ ]:
# Create a colormap based on the "nachfrage" column
vmin = customers_gdf['nachfrage'].min()
vmax = customers_gdf['nachfrage'].max()
cmap = 'coolwarm'  

# Plot the raster with boundary
ax = geo_würzburg.boundary.plot(color='gray', linewidth=0.5, figsize = (5,5))

# Plot the customers with custom colormap
customers_gdf.plot(ax=ax, column='nachfrage', cmap=cmap, vmin=vmin, vmax=vmax)

# Add title and labels
plt.axis('off')
plt.title('Nachfrage der Cluster des Landkreis Würzburg')
#plt.xlabel('Breitengradkoordinate im CSR3035 Format')
#plt.ylabel('Längengradkoordinate im CSR3035 Format')

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []  # Fake up the array of the scalar mappable
cbar = plt.colorbar(sm, ax=ax, shrink=0.5, label='Nachfrage')  # Adjust shrink value as needed

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')  # 1 pixel = 1 meter
ax.add_artist(scalebar)

plt.show()

In [ ]:
# Create a colormap based on the "total_price" column
vmin = warehouses_gdf['total_price_big'].min()
vmax = warehouses_gdf['total_price_big'].max()
cmap = 'coolwarm'  


# Rahmen der Grafik definieren
fig, ax = plt.subplots(figsize=(6, 6))
geo_würzburg.boundary.plot(ax=ax, color='gray', linewidth=0.5)

# Plotte gdf_loaded auf dieselbe Achse
warehouses_gdf.plot(ax=ax, column='total_price_big', cmap=cmap, markersize=20)

# Titel hinzufügen
plt.title('Lagerhaltungsimmobilien (Max) des Landkreis Würzburg')
plt.axis('off')

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []  # Fake up the array of the scalar mappable
cbar = plt.colorbar(sm, ax=ax, shrink=0.5, label='Gesamter Mietpreis für ein Jahr')  # Adjust shrink value as needed

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Zeige den Plot an
plt.show()

In [ ]:
# Create a colormap based on the "total_price" column
vmin = warehouses_gdf['total_price_small'].min()
vmax = warehouses_gdf['total_price_small'].max()
cmap = 'coolwarm'  


# Rahmen der Grafik definieren
fig, ax = plt.subplots(figsize=(6, 6))
geo_würzburg.boundary.plot(ax=ax, color='gray', linewidth=0.5)

# Plotte gdf_loaded auf dieselbe Achse
warehouses_gdf.plot(ax=ax, column='total_price_small', cmap=cmap, markersize=20)

# Titel hinzufügen
plt.title('Lagerhaltungsimmobilien (Min) des Landkreis Würzburg')
plt.axis('off')

# Add colorbar
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []  # Fake up the array of the scalar mappable
cbar = plt.colorbar(sm, ax=ax, shrink=0.5, label='Gesamter Mietpreis für ein Jahr')  # Adjust shrink value as needed

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Zeige den Plot an
plt.show()

In [ ]:
# Convert JSON data to pandas DataFrame
df_energy = pd.DataFrame(data_energy["data"])

# Convert timestamps to datetime format
df_energy["start_timestamp"] = pd.to_datetime(df_energy["start_timestamp"], unit="ms")
df_energy["end_timestamp"] = pd.to_datetime(df_energy["end_timestamp"], unit="ms")

# Filter data for values greater than 0
df_filtered = df_energy[df_energy["marketprice"] > 0]

# Plot market price vs timestamps (using filtered data)
plt.figure(figsize=(12, 6))  # Adjust figure size as needed
plt.plot(df_filtered["start_timestamp"], df_filtered["marketprice"], label="Market Price (Eur/MWh)")

# Set labels and title
plt.xlabel("Date")
plt.ylabel("Market Price (Eur/MWh)")
plt.title("Market Price Fluctuations (Values > 0)")  # Update title

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Set y-axis limits to start from 0 (optional)
plt.ylim(bottom=0)  # This ensures the y-axis starts at 0

# Add gridlines
plt.grid(True)

# Show legend
plt.legend()

plt.tight_layout()
plt.show()

# Optimierungsmodel

## Optimierung

In [ ]:
def optimize(W,
             R,
             S,
             warehouses_gdf_run,
             cost_per_km_drone,
             factory_fix_costs,
             factory_variable_costs,
             factory_operating_costs, 
             qm_per_customer, 
             rent_factor, 
             max_flight_distance, 
             drone_initial_costs, 
             drone_speed, 
             time_window, 
             night_shift_dist, 
             delivery_time,
             alpha_drones,
             customers_gdf_opt):
    
    # Create a solver
    solver = pywraplp.Solver.CreateSolver('GUROBI')

    # Set time limit to a high number (e.g., 1e9 seconds)
    solver.SetTimeLimit(int(1e9))  # Convert to integer

    # Set the absolute MIP gap to 0 and the relative MIP gap to 0
    solver.SetSolverSpecificParametersAsString("MIPGapAbs=0 MIPGap=0")

    # Define decision variables
    # Which warehouse serves which region
    x = {}
    for w, r in S:
        x[w,r] = solver.BoolVar(name=f'x_{w}_{r}')

    # Which warehouses are opened
    y = {}
    # How many drones are needed in each warehouse
    z = {}
    # How much space is rented in each warehouse
    d = {}
    for w in W:
        y[w] = solver.BoolVar(name=f'y_{w}')
        z[w] = solver.IntVar(0, solver.infinity(), name=f'z_{w}')
        d[w] = solver.IntVar(0, solver.infinity(), name=f'd_{w}')
        
    # Objective Function
    objective = solver.Objective()
    
    
    # Fixed costs for opening warehouses
    for w in W:
        objective.SetCoefficient(y[w], factory_fix_costs)  # Add fixed factory setup costs
        objective.SetCoefficient(d[w], factory_variable_costs)  # Add variable factory setup costs
        objective.SetCoefficient(d[w], factory_operating_costs * warehouses_gdf_run.loc[w, 'pricePerSquareMetre'] * 12 * rent_factor)  # Add factory operating costs
        objective.SetCoefficient(d[w], warehouses_gdf_run.loc[w, 'pricePerSquareMetre'] * 12 * rent_factor)  # Cost per square meter

        
    # Costs for acquiring drones
    for w in W:
        objective.SetCoefficient(z[w], drone_initial_costs / 5)
    
    # Variable costs for transportation
    for w, r in S:
        objective.SetCoefficient(x[w,r], cost_per_km_drone * shifts_df.loc[w,r].travel_distance * 2 * customers_gdf_opt.loc[r, 'nachfrage'])

    objective.SetMinimization()
    
    # Constraints
    # Regions can only be served by open warehouses
    for w in W:
        for r in R:
            solver.Add(x[w,r] <= y[w])

    # Each region has to be served by exactly one warehouse
    for r in R:
        solver.Add(solver.Sum(x[w,r] for w in W) == 1)

    # Each warehouse needs to be assigned with a certain amount of drones
    # Definiere das Zeitfenster für die Erfüllung des Demands (in Minuten)
    # Berechne den täglichen Demand Faktor
    daily_demand_factor = (1 - night_shift_dist) / 365

    for w in W:
        # Initialisiere den Ausdruck für die gesamte Reisezeit
        total_time = solver.Sum(
            x[w, r] * shifts_df.loc[w, r].travel_distance * 2 * 
            (customers_gdf_opt.loc[r, 'nachfrage'] * daily_demand_factor / drone_speed)
            for r in R
        ) / time_window

        # Berechne die maximale Anzahl an Drones, die benötigt werden, um parallele oder überlappende Demands zu erfüllen
        max_drones_needed = solver.Sum(
            x[w, r] * shifts_df.loc[w, r].travel_distance * 2 * 
            (customers_gdf_opt.loc[r, 'nachfrage'] * daily_demand_factor / drone_speed)
            for r in R
        ) / delivery_time

        # Berücksichtige eine gewichtete Summe, um beiden Szenarien gerecht zu werden
        combined_drones_needed = total_time + alpha_drones * (max_drones_needed - total_time)

        # Füge die erweiterte Constraint hinzu
        solver.Add(combined_drones_needed <= z[w])


    # Each warehouse is assigned a certain amound of space that is between the boundries of the offering
    for w in W:
        solver.Add(y[w] * warehouses_gdf_run.loc[w, 'floorSpace_small'] <= d[w])
        solver.Add(y[w] * warehouses_gdf_run.loc[w, 'floorSpace_big'] >= d[w])
        
    # The distance from warehouse to customer can't be taller than the maximum flight range of each drone
    for w in W:
        for r in R:
            solver.Add(x[w,r] * shifts_df.loc[w,r].travel_distance <= max_flight_distance)

    # Each warehouse needs a certain amount of space for each customer served
    for w in W:
        customer_demand_sum = solver.Sum(x[w, r] * (customers_gdf_opt.loc[r, 'nachfrage'] / 365) for r in R)
        required_space_for_customers = customer_demand_sum * qm_per_customer
        solver.Add(d[w] >= required_space_for_customers)

    return solver, x, y, z, d

## Lösungsausgabe

In [ ]:
def solve(W, R, solver, x, y, z, d, customers_gdf_run, warehouses_gdf_run):
  opened_warehouses = []
  customers_gdf_run['assigned_warehouse'] = 0
  warehouses_gdf_run['number_of_drones'] = 0
  warehouses_gdf_run['floor_space_assigned'] = 0

  #Solving the problem
  status = solver.Solve()
  print('Solved!')

  def print_solution(status, solver, opened_warehouses, x, y, z, d, W, R):

    if status == pywraplp.Solver.OPTIMAL:
      print("Objective value:", solver.Objective().Value())
      #print("Opened warehouses:")
      opened_warehouses.clear()  # Clear the list before appending
      for w in W:
        if y[w].solution_value() > 0.5:
          opened_warehouses.append(w)
          warehouses_gdf_run.loc[w, 'number_of_drones'] = z[w].solution_value()
          warehouses_gdf_run.loc[w, 'floor_space_assigned'] = d[w].solution_value()
          # print(f"- Warehouse {w}")
          # print(f"Floor-Space: {d[w].solution_value()}")
          # print(f"Drones needed: {z[w].solution_value()}")
      #print("Warehouse assignments:")
      for r in R:
        assigned_warehouse = None
        for w in W:
          if x[w, r].solution_value() > 0.5:
            assigned_warehouse = w
            break
        if assigned_warehouse is not None:
          customers_gdf_run.loc[r, 'assigned_warehouse'] = assigned_warehouse
          #print(f"- Region {r} served by warehouse {assigned_warehouse}")
          #distance = shifts_df.loc[assigned_warehouse, r].travel_distance
          #print(f"- Distance: {distance}")
        else:
          print(f"- Region {r} has no assigned warehouse (might be infeasible)")
    else:
      print("Solver failed to find an optimal solution. Status:", status)


  print_solution(status, solver, opened_warehouses, x, y, z, d, W, R)
  print(f'Opened Warehouses: {opened_warehouses}')
  
  return opened_warehouses, customers_gdf_run, warehouses_gdf_run

## Solution Graphics

In [ ]:
#Rahmen der Grafik definieren
ax = geo_würzburg.boundary.plot(color='gray', linewidth=0.5, figsize = (7,7))

# Plotte gdf_loaded auf dieselbe Achse
warehouses_gdf[~warehouses_gdf.index.isin(opened_warehouses)].plot(ax=ax, color='red', marker='o', markersize=25)
warehouses_gdf.loc[opened_warehouses].plot(ax=ax, color='blue', marker='^', markersize=35)

# Titel hinzufügen
plt.title('Lagerhaltungsimmobilien des Landkreis Würzburg')

plt.axis('off')

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Add legend
custom_legend = [Line2D([0], [0], marker='^', color='w', markerfacecolor='blue', markersize=10, label='Geöffnet'),
                 Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Nicht geöffnet')]
plt.legend(handles=custom_legend, title='Warenhäuser', bbox_to_anchor=(1,0.95), loc='upper left', borderaxespad=0., fontsize='small', ncol=1)
plt.tight_layout()

# # Index jedes Warehouses anzeigen
# for idx, row in warehouses_gdf.loc[opened_warehouses].iterrows():
#     plt.annotate(idx, (row.geometry.x, row.geometry.y), xytext=(5, 5), textcoords='offset points', fontsize=8, color='black')

# for idx, row in warehouses_gdf[~warehouses_gdf.index.isin(opened_warehouses)].iterrows():
#     plt.annotate(idx, (row.geometry.x, row.geometry.y), xytext=(5, 5), textcoords='offset points', fontsize=8, color='black')

# Zeige den Plot an
plt.show()

In [ ]:
# Create a colormap with a color for each opened warehouse
colors = plt.cm.tab20(np.linspace(0, 1, len(opened_warehouses)))
warehouse_colors = dict(zip(opened_warehouses, colors))

# Create a dictionary to map each warehouse to a color
warehouse_colors = {
    warehouse: color
    for warehouse, color in zip(opened_warehouses, warehouse_colors.values())
}

# Plot the boundaries of the region
ax = geo_würzburg.boundary.plot(color='gray', linewidth=0.5, figsize=(8, 8))


# Plot the customers with the color of their assigned warehouse
for index, row in customers_gdf.iterrows():
    warehouse = row['assigned_warehouse']
    color = warehouse_colors.get(warehouse, 'gray')  # Use gray color if warehouse is not in the dictionary
    customers_gdf.iloc[[index]].plot(ax=ax, color=color, markersize=20)


# Plot the warehouses with their assigned colors
for warehouse, color in warehouse_colors.items():
    warehouses_gdf[warehouses_gdf.index == warehouse].plot(ax=ax, color=color, marker='^', markersize=50, edgecolor='black')


# Turn off axis
plt.axis('off')

# Add title
plt.title('Lagerhaltungsimmobilien des Landkreis Würzburg')

# Add legend
legend_handles = [Line2D([0], [0], marker='^', color='w', markerfacecolor=color, markersize=10, label=warehouse) for warehouse, color in warehouse_colors.items()]
ax.legend(handles=legend_handles, title='Warenhäuser', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., fontsize='small', ncol=2)

# Add scale bar
scalebar = ScaleBar(1, location='lower left', label='Maßstab')
ax.add_artist(scalebar)

# Show the plot
plt.show()

## Assigning a warehouse to each building based on the optimal value

In [ ]:
# Funktion, um das zugewiesene Lager für einen Punkt zu finden
def find_assigned_warehouse(point, customers_gdf_run, cluster_sindex):
    # Räumlichen Index für das Cluster-GDF erstellen
    possible_matches_index = list(cluster_sindex.intersection(point.bounds))
    possible_matches = customers_gdf_run.iloc[possible_matches_index]
    output = possible_matches[possible_matches.geometry.contains(point)]
    if not output.empty:
        return [output.assigned_warehouse.iloc[0]]
    else:
        nearest_polygon_index = cluster_sindex.nearest(point)[0]
        nearest_polygon = customers_gdf_run.iloc[nearest_polygon_index]
        return [nearest_polygon.assigned_warehouse.iloc[0]]

In [ ]:
def assign_warehouses(bevölkerungs_gdf_run, customers_gdf_run, warehouses_gdf_run):
    # Verfolgen Sie den Fortschritt der apply-Methode
    tqdm.pandas()
    
    cluster_sindex = customers_gdf_run.sindex

    # Die apply-Methode auf die GeoDataFrame anwenden, um das zugewiesene Lager für jeden Punkt zu finden
    warehouses = bevölkerungs_gdf_run['geometry'].progress_apply(find_assigned_warehouse, customers_gdf_run = customers_gdf_run, cluster_sindex = cluster_sindex)

    bevölkerungs_gdf_run['assigned_warehouse'] = 0
    bevölkerungs_gdf_run['distance_warehouse'] = 0.0

    for index, row in tqdm(bevölkerungs_gdf_run.iterrows(), total=len(bevölkerungs_gdf_run)):
        bevölkerungs_gdf_run.loc[index, 'assigned_warehouse'] = warehouses[index][0]
        warehouse_geometry = warehouses_gdf_run.loc[row['assigned_warehouse'], 'geometry']
        population_geometry = row['geometry']
        bevölkerungs_gdf_run.loc[index, 'distance_warehouse'] = warehouse_geometry.distance(population_geometry) / 1000

    return bevölkerungs_gdf_run

## Simulations LoopFunctions

In [ ]:
def calculate_demand_sim(row, demand_factor):
    
    probabilities = {
        '0-10': 0,
        '10-20': 0.003458068783068975,
        '20-30': 0.013896825396825975,
        '30-40': 0.02136825396825512,
        '40-50': 0.02338333333333506,
        '50-60': 0.026277248677250214,
        '60-70': 0.03332089947090043,
        '70-80': 0.046515343915346036,
        '80+': 0.046515343915346036,
    }
    
    total_demand = 0
    for age_group, count in row['Alter'].items():
        lambda_value = count * probabilities[age_group]
        total_demand += poisson.rvs(lambda_value)
    return int(total_demand * demand_factor)

In [ ]:
def assign_random_timestamp(row, night_shift_dist, start_time, end_time):
    timestamp_list = []
    for i in range(row['nachfrage']): 
        probabilities_timestamp = [1 - night_shift_dist, night_shift_dist]  # Wahrscheinlichkeit für innerhalb und außerhalb der Öffnungszeiten

        start_hour = start_time.hour
        start_minute = start_time.minute
        end_hour = end_time.hour
        end_minute = end_time.minute

        if np.random.choice([False, True], p=probabilities_timestamp):
            # Außerhalb der Öffnungszeiten
            if random.choice([True, False]):
                # Vor den Öffnungszeiten
                hour = random.randint(0, start_hour - 1)
                minute = random.randint(0, 59)
                second = random.randint(0, 59)
            else:
                # Nach den Öffnungszeiten
                hour = random.randint(end_hour + 1, 23)
                minute = random.randint(0, 59)
                second = random.randint(0, 59)
        else:
            # Innerhalb der Öffnungszeiten
            if start_hour == end_hour:
                hour = start_hour
                minute = random.randint(start_minute, end_minute)
            else:
                hour = random.randint(start_hour, end_hour)
                if hour == start_hour:
                    minute = random.randint(start_minute, 59)
                elif hour == end_hour:
                    minute = random.randint(0, end_minute)
                else:
                    minute = random.randint(0, 59)
            second = random.randint(0, 59)
        
        timestamp = datetime.combine(datetime.today(), datetime.min.time()) + timedelta(hours=hour, minutes=minute, seconds=second)
        timestamp_list.append(timestamp)
    return timestamp_list

In [ ]:
def calculate_penalty_costs(bevoelkerungs_gdf_sim, warehouses_gdf_sim, drone_speed, time_window, opening_start_time, opening_end_time):
    # Convert time_window to a timedelta object
    time_window = timedelta(minutes=time_window)
    
    # Initialize a dictionary to keep track of next available times for drones in each warehouse
    warehouse_drones = {warehouse: [opening_start_time] * warehouses_gdf_sim.loc[warehouse, 'number_of_drones'] for warehouse in warehouses_gdf_sim.index}
    
    total_exceeded_minutes = 0
    number_of_trips_exceeded = 0
    delivery_minutes_needed = []

    # Flatten the demand timestamps and associate them with their warehouses and distances
    all_demands = []
    for index, row in bevoelkerungs_gdf_sim.iterrows():
        assigned_warehouse = row['assigned_warehouse']
        distance_warehouse = row['distance_warehouse']
        for timestamp in row['demand_timestamp']:
            all_demands.append((assigned_warehouse, distance_warehouse, timestamp))
    
    # Sort all demands by timestamp
    all_demands.sort(key=lambda x: x[2])
    
    def time_to_minutes(t):
        """Convert a time object to minutes since midnight."""
        return t.hour * 60 + t.minute
    
    def minutes_to_time(m):
        """Convert minutes since midnight to a time object."""
        return time(int(m // 60), int(m % 60))
    
    opening_start_minutes = time_to_minutes(opening_start_time)
    opening_end_minutes = time_to_minutes(opening_end_time)
    
    # Process each demand
    for assigned_warehouse, distance_warehouse, timestamp in all_demands:
        
        # Find the first available drone
        available_drones = warehouse_drones[assigned_warehouse]
        next_available_time = min(available_drones, key=time_to_minutes)
        start_time = max(time_to_minutes(timestamp), time_to_minutes(next_available_time))  # Use start_time to calculate when the drone can actually start
        
        # Ensure start time is within opening hours
        if start_time < opening_start_minutes:
            start_time = opening_start_minutes
        elif start_time > opening_end_minutes:
            continue  # Skip demands outside of opening hours

        # Calculate the delivery time
        delivery_time = start_time + int((2 * distance_warehouse) / (drone_speed))  # Round trip time in minutes
        delivery_minutes_needed.append(delivery_time - time_to_minutes(timestamp))
        
        # Ensure delivery time is within opening hours
        if delivery_time > opening_end_minutes:
            continue  # Skip deliveries that cannot be completed within opening hours

        # Check if the delivery time exceeds the time window
        if (delivery_time - time_to_minutes(timestamp)) > time_window.total_seconds() / 60:
            exceeded_minutes = (delivery_time - time_to_minutes(timestamp)) - (time_window.total_seconds() / 60)
            total_exceeded_minutes += exceeded_minutes
            number_of_trips_exceeded +=1

        # Update the next available time for the drone
        return_trip_end_time = minutes_to_time(delivery_time)
        available_drones[available_drones.index(next_available_time)] = return_trip_end_time

    if number_of_trips_exceeded == 0:
        return 0, np.median(delivery_minutes_needed)
    
    return total_exceeded_minutes / number_of_trips_exceeded, np.median(delivery_minutes_needed)

In [ ]:
def build_trip_request_string(pharmacy_lon, pharmacy_lat, demands):
    waypoints = f"{pharmacy_lon},{pharmacy_lat}"
    for _, row in demands.iterrows():
        waypoints += f";{row.lon},{row.lat}"
    return f"http://router.project-osrm.org/trip/v1/driving/{waypoints}?roundtrip=true&source=first&destination=last&overview=false&steps=false"

def calculate_total_distance_and_duration(demands, pharmacy_lon, pharmacy_lat, total_distance_sum, total_duration_sum, counter):
    try:
        request_string = build_trip_request_string(pharmacy_lon, pharmacy_lat, demands)
        res = requests.get(request_string)
        res.raise_for_status()  # Raise an exception for non-200 status codes

        content = json.loads(res.content)

        # Check if trips are available
        if 'trips' in content and len(content['trips']) > 0:
            trip = content['trips'][0]
            total_distance = trip['distance']
            total_duration = trip['duration']
            return total_distance, total_duration
        else:
            #print(f"No trips found for request {request_string}")
            return total_distance_sum/counter, total_duration_sum/counter

    except requests.exceptions.RequestException as e:
        #print(f"Error occurred on attempt for request {request_string}: {e}")
        return total_distance_sum/counter, total_duration_sum/counter

def expand_timestamps(demands_df):
    # Explode the DataFrame by demand_timestamp
    demands_df = demands_df.explode('demand_timestamp')
    demands_df['demand_timestamp'] = pd.to_datetime(demands_df['demand_timestamp'])
    return demands_df

def calculate_pharmacy_routing(demands_df, pharmacies_df):
    # Expand the demands DataFrame so each timestamp is in its own row
    demands_df = expand_timestamps(demands_df)

    # Split demands into two DataFrames by half, grouped by assigned pharmacy
    demands_df_groups = demands_df.groupby('assigned_pharmacy')
    groups_list = list(demands_df_groups)
    split_index = len(groups_list) // 2

    # Split the DataFrame into two DataFrames
    first_half_demands = pd.concat([group[1] for group in groups_list[:split_index]])
    second_half_demands = pd.concat([group[1] for group in groups_list[split_index:]])
    

    # Group by assigned pharmacy and filter demands by time
    before_13pm = first_half_demands[first_half_demands['demand_timestamp'].dt.hour < 13].drop_duplicates(subset=['lon', 'lat'])
    after_13pm = first_half_demands[first_half_demands['demand_timestamp'].dt.hour >= 13].drop_duplicates(subset=['lon', 'lat'])

    total_distance_sum = 0
    total_duration_sum = 0
    counter = 1

    # Vectorized processing within each time period
    for time_period, demands in [('before_13pm', before_13pm), ('after_13pm', after_13pm)]:
        if demands.empty:
            continue

        grouped = demands.groupby('assigned_pharmacy')

        for pharmacy_id, group in grouped:
            pharmacy_info = pharmacies_df[pharmacies_df['id'] == pharmacy_id]

            if pharmacy_info.empty:
                print(f"Pharmacy ID {pharmacy_id} not found in pharmacies dataframe.")
                continue

            pharmacy_lon = pharmacy_info.lon.iloc[0]
            pharmacy_lat = pharmacy_info.lat.iloc[0]

            total_distance, total_duration = calculate_total_distance_and_duration(group, pharmacy_lon, pharmacy_lat, total_distance_sum, total_duration_sum, counter)
            total_distance_sum += total_distance
            total_duration_sum += total_duration
            counter += 1
            

    
    second_half_demands = second_half_demands.groupby('assigned_pharmacy')
    for pharmacy_id, group in second_half_demands:
        pharmacy_info = pharmacies_df[pharmacies_df['id'] == pharmacy_id]

        if pharmacy_info.empty:
            print(f"Pharmacy ID {pharmacy_id} not found in pharmacies dataframe.")
            continue

        pharmacy_lon = pharmacy_info.lon.iloc[0]
        pharmacy_lat = pharmacy_info.lat.iloc[0]

        total_distance, total_duration = calculate_total_distance_and_duration(group, pharmacy_lon, pharmacy_lat, total_distance_sum, total_duration_sum, counter)
        total_distance_sum += total_distance
        total_duration_sum += total_duration
        counter += 1

    return total_distance_sum / 1000, total_duration_sum / 60 #Meters to km, seconds to minutes

## Simulation

In [ ]:
def simulate(opened_warehouses_run, bevölkerungs_gdf_run, warehouses_gdf_run, cost_per_km_drone, drone_speed, delivery_time, start_time, end_time, demand_factor, rent_factor, night_shift_dist, cost_per_km_car, cost_per_km_truck, factory_fix_costs, factory_variable_costs, factory_operating_costs, drone_initial_costs, sensitivity_run):

    #Calculate the monthly fix costs due to drone and factory setup
    rental_cost = (np.sum(warehouses_gdf_run.loc[opened_warehouses_run].floor_space_assigned * warehouses_gdf_run.loc[opened_warehouses_run].pricePerSquareMetre)) * rent_factor
    factory_fix_monthly = (len(opened_warehouses_run) * factory_fix_costs) / 12
    factory_variable_monthly = np.sum(warehouses_gdf_run.loc[opened_warehouses_run].floor_space_assigned * factory_variable_costs) / 12
    factory_operating_monthly = np.sum(warehouses_gdf_run.loc[opened_warehouses_run].floor_space_assigned * warehouses_gdf_run.loc[opened_warehouses_run].pricePerSquareMetre * factory_operating_costs)
    drone_setup_cost = ((np.sum(warehouses_gdf_run.loc[opened_warehouses_run].number_of_drones) * drone_initial_costs) / 12) / 5

    drone_transportation_cost = 0
    drone_transportation_time = 0
    median_penalty_time = 0
    median_waiting_time = 0
    avg_penalty_time = []
    avg_waiting_time = []

    car_transportation_cost_customer = 0
    car_transportation_time_customer = 0

    car_transportation_cost_pharmacy = 0
    car_transportation_time_pharmacy = 0


    tqdm.pandas()

    #Loop der Simulation über ein gesamtes Jahr
    for i in range(30):

        print(f'Simulation - Tag: {i + 1}')
        
        bevölkerungs_gdf_run['nachfrage'] = bevölkerungs_gdf_run.progress_apply(calculate_demand_sim, axis=1, demand_factor = demand_factor)
        bevölkerungs_gdf_run['demand_timestamp'] = bevölkerungs_gdf_run.progress_apply(lambda row: assign_random_timestamp(row, night_shift_dist, start_time, end_time), axis=1)

        # Calculate transportation costs and time using vectorized operations
        drone_transportation_cost += np.sum(bevölkerungs_gdf_run['nachfrage'] * bevölkerungs_gdf_run['distance_warehouse'] * cost_per_km_drone * 2)
        drone_transportation_time += np.sum((bevölkerungs_gdf_run['nachfrage'] * bevölkerungs_gdf_run['distance_warehouse']) / drone_speed )
        median_penalty_time, median_waiting_time  = calculate_penalty_costs(bevölkerungs_gdf_run, warehouses_gdf_run, drone_speed, delivery_time, start_time, end_time)
        avg_penalty_time.append(median_penalty_time)
        avg_waiting_time.append(median_waiting_time)

        if sensitivity_run:
            car_transportation_cost_customer += np.sum(bevölkerungs_gdf_run['nachfrage'] * bevölkerungs_gdf_run['distance_pharmacy'] * cost_per_km_car * 2)
            car_transportation_time_customer += np.sum(bevölkerungs_gdf_run['nachfrage'] * bevölkerungs_gdf_run['time_pharmacy'])

            car_transportation_cost_pharmacy_temp, car_transportation_time_pharmacy_temp = calculate_pharmacy_routing(bevölkerungs_gdf_run[bevölkerungs_gdf_run['nachfrage'] > 0], pharmacy_gdf)
            car_transportation_cost_pharmacy += car_transportation_cost_pharmacy_temp * cost_per_km_truck + bevölkerungs_gdf_run['distance_pharmacy'].median() * cost_per_km_truck
            car_transportation_time_pharmacy += car_transportation_time_pharmacy_temp
        
        bevölkerungs_gdf_run.drop('nachfrage', axis=1, inplace=True)
        bevölkerungs_gdf_run.drop('demand_timestamp', axis=1, inplace=True)

    return [rental_cost, factory_fix_monthly, factory_variable_monthly, factory_operating_monthly ,drone_setup_cost, drone_transportation_cost, drone_transportation_time, np.median(avg_penalty_time), np.median(avg_waiting_time), car_transportation_cost_customer, car_transportation_time_customer, car_transportation_cost_pharmacy, car_transportation_time_pharmacy]

In [ ]:
def print_cost_summary(costs):

  print("-" * 50)
  print("Logistical Cost Summary (per year):")
  print("-" * 50)
  print(f"Factory Rental Cost: \t\t\t€{costs[0]:.2f}")
  print(f"Factory Fix Cost: \t\t\t€{costs[1]:.2f}")
  print(f"Factory Variable Cost: \t\t\t€{costs[2]:.2f}")
  print(f"Factory Operating Cost: \t\t€{costs[3]:.2f}")
  print(f"Drone Setup Cost: \t\t\t€{costs[4]:.2f}")
  print("-" * 50)
  print(f"Drone Transportation Cost: \t\t€{costs[5]:.2f}")
  print(f"Drone Transportation Time: \t\t{costs[6]/60:.2f} hours")
  print(f"Median Penalty Time: \t{costs[7]:.2f}")
  print(f"Median Waiting Time: \t{costs[8]:.2f}")
  print("-" * 50)
  print(f"Car/Truck Customer Transportation Cost: €{costs[9]:.2f}")
  print(f"Car/Truck Customer Transportation Time: {costs[10]/60:.2f} hours")
  print("-" * 50)
  print(f"Car/Truck Pharmacy Transportation Cost: €{costs[11]:.2f}")
  print(f"Car/Truck Pharmacy Transportation Time: {costs[12]/60:.2f} hours")
  print("-" * 50)

# Robustheit / Parameter Changes

## Run the sim

In [ ]:
def calculate_times(minutes):
  start_time = time(hour=8, minute=0) # Set initial start time to 8:00 AM
  total_minutes = (start_time.hour * 60 + start_time.minute) + minutes # Convert minutes to total number of minutes
  end_time = time(hour=total_minutes // 60 % 24, minute=total_minutes % 60)   # Calculate end time by handling overflow within 24 hours

  # Adjust start time if end time is before start time (overflow)
  if end_time < start_time:
    # Calculate the adjustment needed (difference in minutes)
    if start_time.minute > end_time.minute:
      start_time = time(start_time.hour - end_time.hour, start_time.minute - end_time.minute)
    else:
      start_time = time(start_time.hour - end_time.hour -1, 60 - end_time.minute)
    end_time = time(23,59,59)

  return start_time, end_time

In [ ]:
def run(parameter_values, bevölkerungs_gdf_run, warehouses_gdf_run, customers_gdf_run, shifts_df_run, optimal_run, city, opened_warehouses_optimal, sensitivity_run):

    # Drone Parameters
    cost_per_km_drone = (parameter_values['kwh_eur'] * parameter_values['watt_drone']) / (parameter_values['drone_speed'] * 60)  # EUR/km # Calculate cost per kilometer for the drone
    # Car/Truck Parameters:
    cost_per_km_car = 0.5
    cost_per_km_truck = 2

    W = warehouses_gdf_run.index.values
    R = customers_gdf_run.index.values
    S = shifts_df_run.index.values
    
    if optimal_run:
        # Set the optimization problem
        solver, x, y, z, d = optimize(
            W = W, 
            R = R,
            S = S, 
            warehouses_gdf_run = warehouses_gdf_run,
            customers_gdf_opt = customers_gdf_run, 
            cost_per_km_drone = cost_per_km_drone,
            factory_fix_costs = parameter_values['factory_fix_costs'],
            factory_variable_costs = parameter_values['factory_variable_costs'],
            factory_operating_costs =  parameter_values['factory_operating_costs'],
            qm_per_customer = parameter_values['qm_per_customer'],
            rent_factor = parameter_values['rent_factor'],
            max_flight_distance = parameter_values['max_flight_distance'],
            drone_initial_costs = parameter_values['drone_initial_costs'],
            drone_speed = parameter_values['drone_speed'],
            time_window = parameter_values['time_window'],
            alpha_drones = parameter_values['alpha'],
            delivery_time = parameter_values['delivery_time'],
            night_shift_dist = parameter_values['night_shift_dist']
            )

        print('Solver set up!')

        # Solve the problem and get the solution
        opened_warehouses, customers_gdf_run, warehouses_gdf_run = solve(
            W = W,
            R = R, 
            solver = solver, 
            x = x, 
            y = y, 
            z = z, 
            d = d, 
            customers_gdf_run = customers_gdf_run, 
            warehouses_gdf_run = warehouses_gdf_run)
        
        # Assign the in the solution chosen warehouses in the dataset
        bevölkerungs_gdf_run = assign_warehouses(bevölkerungs_gdf_run=bevölkerungs_gdf_run,
                                                 customers_gdf_run=customers_gdf_run,
                                                 warehouses_gdf_run=warehouses_gdf_run)
        print('Dataset set up!')

    
    if not optimal_run: 
        opened_warehouses = opened_warehouses_optimal

    # Get the start and end time of the service hours
    start_time, end_time = calculate_times(minutes=parameter_values['time_window'])



    # Simulate with optimal values
    costs = simulate(
        opened_warehouses_run = opened_warehouses, 
        bevölkerungs_gdf_run = bevölkerungs_gdf_run, 
        warehouses_gdf_run = warehouses_gdf_run, 
        cost_per_km_drone = cost_per_km_drone,
        start_time = start_time,
        end_time = end_time,
        demand_factor = parameter_values['demand_factor'],
        rent_factor = parameter_values['rent_factor'],
        drone_speed = parameter_values['drone_speed'],
        delivery_time = parameter_values['delivery_time'],
        night_shift_dist = parameter_values['night_shift_dist'],
        cost_per_km_car = cost_per_km_car,
        cost_per_km_truck = cost_per_km_truck,
        factory_fix_costs = parameter_values['factory_fix_costs'],
        factory_variable_costs = parameter_values['factory_variable_costs'],
        factory_operating_costs =  parameter_values['factory_operating_costs'],
        drone_initial_costs = parameter_values['drone_initial_costs'],
        sensitivity_run = sensitivity_run) 
    print('Simulation done!')

    print_cost_summary(costs = costs)


    if optimal_run:
        if not sensitivity_run:
            customers_gdf_run['Alter'] = customers_gdf_run['Alter'].apply(json.dumps)
            customers_gdf_run.to_file(f'./Results/Optimal_Results_customers_{city}.gpkg', driver = 'GPKG')

            bevölkerungs_gdf_run['Alter'] = bevölkerungs_gdf_run['Alter'].apply(json.dumps)
            bevölkerungs_gdf_run['Geschlecht'] = bevölkerungs_gdf_run['Geschlecht'].apply(json.dumps)
            bevölkerungs_gdf_run.to_file(f'./Results/Optimal_Results_bevoelkerung_{city}.gpkg', driver = 'GPKG')

    
            warehouses_gdf_run.to_file(f'./Results/Optimal_Results_warehouses_{city}.gpkg', driver = 'GPKG')

        
        # Dictionary erstellen, um die Werte zu speichern
        warehouse_info = {}
        # Durch die geöffneten Lagerhäuser iterieren und die entsprechenden Werte hinzufügen
        for warehouse_id in opened_warehouses:
            if warehouse_id in warehouses_gdf_run.index:
                warehouse_info[warehouse_id] = {
                    'floor_space_assigned': warehouses_gdf_run.at[warehouse_id, 'floor_space_assigned'],
                    'number_of_drones': warehouses_gdf_run.at[warehouse_id, 'number_of_drones']
        }

        return {
                'opened_warehouses': np.array(opened_warehouses),
                'number_of_drones': np.sum(warehouses_gdf_run.loc[opened_warehouses].number_of_drones),
                'floor_space_assigned': np.sum(warehouses_gdf_run.loc[opened_warehouses].floor_space_assigned),
                'objective_value': solver.Objective().Value(),
                'Factory Rental Cost': costs[0],
                'Factory Fix Cost': costs[1],
                'Factory Variable Cost': costs[2],
                'Factory Operating Cost': costs[3],
                'Drone Setup Cost': costs[4],
                'Drone Transportation Cost': costs[5],
                'Drone Transportation Time': costs[6]/60,
                'Transportation Penalty Cost': costs[7],
                'Transportation Avg Waiting Time': costs[8],
                'Car/Truck Customer Transportation Cost': costs[9],
                'Car/Truck Customer Transportation Time': costs[10],
                'Car/Truck Pharmacy Transportation Cost': costs[11],
                'Car/Truck Pharmacy Transportation Time': costs[12],
                'Warehouse Info': warehouse_info
                }
    return {
        'opened_warehouses': np.array(opened_warehouses_optimal),
        'number_of_drones': np.sum(warehouses_gdf_run.loc[opened_warehouses_optimal].number_of_drones),
        'floor_space_assigned': np.sum(warehouses_gdf_run.loc[opened_warehouses_optimal].floor_space_assigned),
        'Factory Rental Cost': costs[0],
        'Factory Fix Cost': costs[1],
        'Factory Variable Cost': costs[2],
        'Factory Operating Cost': costs[3],
        'Drone Setup Cost': costs[4],
        'Drone Transportation Cost': costs[5],
        'Drone Transportation Time': costs[6],
        'Transportation Penalty Cost': costs[7],
        'Transportation Avg Waiting Time': costs[8],
        'Car/Truck Customer Transportation Cost': costs[9],
        'Car/Truck Customer Transportation Time': costs[10],
        'Car/Truck Pharmacy Transportation Cost': costs[11],
        'Car/Truck Pharmacy Transportation Time': costs[12]
        }

## Sensitivity Analysis

In [ ]:
def sensitivity_analysis(parameter_values, number_of_runs, sensitivity_index, parameter_names, optimal_run, sensitivity_run, bevölkerungs_gdf_run, warehouses_gdf_run, customers_gdf_run, shifts_df_run, opened_warehouses_optimal, warehouses_gdf_optimal, customers_gdf_optimal, bevölkerungs_gdf_optimal, city):
    
    if sensitivity_index != 'None':
        triangular_values = np.random.triangular(parameter_values[sensitivity_index][0], parameter_values[sensitivity_index][1], parameter_values[sensitivity_index][2], number_of_runs)
        triangular_values = np.append(triangular_values, parameter_values[sensitivity_index][0])
        triangular_values = np.append(triangular_values, parameter_values[sensitivity_index][2])

    result_list = []

    for i in range(number_of_runs):
    
        if sensitivity_index != 'None':
            parameter_values[sensitivity_index][1] = triangular_values[i]

        base_values = [param[1] for param in parameter_values]

        # Set the demand in case of demand_factor
        customers_gdf_run_demand = customers_gdf_run.copy()
        customers_gdf_run_demand['nachfrage'] = customers_gdf_run_demand['nachfrage'] * base_values[13]

        warehouses_gdf_filtered = warehouses_gdf_run[warehouses_gdf_run['floorSpace_small'] >= base_values[4]].copy()

        base_dict = {
            'factory_fix_costs': base_values[0],
            'factory_variable_costs': base_values[1],
            'factory_operating_costs': base_values[2],
            'qm_per_customer': base_values[3],
            'minimum_square_requirement': base_values[4],
            'rent_factor': base_values[5],
            'max_flight_distance': base_values[6],
            'drone_initial_costs': base_values[7],
            'drone_speed': base_values[8],
            'time_window': base_values[9],
            'delivery_time': base_values[10],
            'alpha': base_values[11],
            'night_shift_dist': base_values[12],
            'demand_factor': base_values[13],
            'watt_drone': base_values[14],
            'kwh_eur': base_values[15]
            }

        if optimal_run:
            results = run(
                parameter_values = base_dict, 
                bevölkerungs_gdf_run = bevölkerungs_gdf_run, 
                warehouses_gdf_run = warehouses_gdf_filtered, 
                customers_gdf_run = customers_gdf_run_demand, 
                shifts_df_run = shifts_df_run, 
                optimal_run = True,
                opened_warehouses_optimal = None,
                city = city,
                sensitivity_run = False
                )
        
        else: 
            results_new_optimal = run(
                parameter_values = base_dict, 
                bevölkerungs_gdf_run = bevölkerungs_gdf_run, 
                warehouses_gdf_run = warehouses_gdf_filtered, 
                customers_gdf_run = customers_gdf_run_demand, 
                shifts_df_run = shifts_df_run, 
                optimal_run = True, 
                opened_warehouses_optimal = opened_warehouses_optimal,
                city = city,
                sensitivity_run = sensitivity_run
                )
            
            results_old_optimal = run(
                    parameter_values = base_dict, 
                    bevölkerungs_gdf_run = bevölkerungs_gdf_optimal, 
                    warehouses_gdf_run = warehouses_gdf_optimal, 
                    customers_gdf_run = customers_gdf_optimal, 
                    shifts_df_run = shifts_df_run, 
                    optimal_run = False, 
                    opened_warehouses_optimal = opened_warehouses_optimal,
                    city = city,
                    sensitivity_run = sensitivity_run
                    )
        
        if optimal_run:
            result_list.append({
                'parameter': 'Base Value',
                'parameter_value': 'Base_Value',
                'results': results
            })
  
        else: 
            result_list.append({
                'parameter': parameter_names[sensitivity_index],
                'parameter_value': triangular_values[i],
                'results_old_optimal': results_old_optimal,
                'results_new_optimal': results_new_optimal
            })

    
    return result_list

# Complete Loop

## Data Setup

In [ ]:
def find_min_travel_distance(shifts_df):
    # Initialize variables
    min_distance = 0
    max_distance = shifts_df['travel_distance'].max()
    shifts_df.reset_index(inplace=True)

    while min_distance <= max_distance:

        current_distance = min_distance + 1

        # Assuming 'node_a' and 'node_b' are columns containing node IDs, and 'distance' is the distance column
        filtered_data = shifts_df[shifts_df['travel_distance'] < current_distance]

        all_regions = shifts_df.region_id.unique()
        filtered_regions = filtered_data.region_id.unique()  
        region_present = set(all_regions).issubset(set(filtered_regions))

        if region_present:
            break
        else:
            min_distance = current_distance  # Update min distance if condition not met

    shifts_df.set_index(['warehouse_id', 'region_id'], inplace=True)
    return min_distance  # Minimum distance found

In [ ]:
parameter_names = [
    "factory_fix_costs",
    "factory_variable_costs",
    "factory_operating_costs",
    "qm_per_customer",
    "minimum_square_requirement",
    "rent_factor",
    "max_flight_distance",
    "drone_initial_costs",
    "drone_speed",
    "time_window",
    "delivery_time",
    "alpha",
    "night_shift_dist",
    "demand_factor",
    "watt_drone",
    "kwh_eur"
]

In [ ]:
def setup_parameters(shifts_df): 
    base_factory_fix_costs = 50000
    min_factory_fix_costs = 10000
    max_factory_fix_costs = 100000

    base_factory_variable_costs = 0.5
    min_factory_variable_costs = 0.25 
    max_factory_variable_costs = 0.75 

    base_factory_operating_costs = 0.375
    min_factory_operating_costs = 0.25 
    max_factory_operating_costs = 0.5 

    base_qm_per_customer = 0.5 #Quadratmeter
    min_qm_per_customer = 0.25
    max_qm_per_customer = 0.75

    base_minimum_square_requirement = 750 #Quadratmeter
    min_minimum_square_requirement = 500
    max_minimum_square_requirement = 1000 

    base_rent_factor = 1
    min_rent_factor = 0.5 
    max_rent_factor = 1.5 

    base_max_flight_distance = 20 #km
    min_max_flight_distance = find_min_travel_distance(shifts_df=shifts_df)
    max_max_flight_distance = 40 

    base_drone_initial_costs = 4000 #Eur
    min_drone_initial_costs = 2000 
    max_drone_initial_costs = 6000 

    base_drone_speed = 65/60 #km/h/60 = km/min
    min_drone_speed = 50/60 
    max_drone_speed = 80/60 

    base_time_window = 630 #Min
    min_time_window = 60 
    max_time_window = 1439 

    base_delivery_time = 60 #Min
    min_delivery_time = 30
    max_delivery_time = 90

    base_alpha = 0.3
    min_alpha = 0.15
    max_alpha = 0.45

    base_night_shift_dist = 0.00112103746
    min_night_shift_dist = 0.00056051873
    max_night_shift_dist = 0.00168155619 

    base_demand_factor = 1 #Prozent
    min_demand_factor = 0.5 
    max_demand_factor = 1.5 

    base_watt_drone = 1.08 #Kw
    min_watt_drone = 0.85 
    max_watt_drone = 1.2 

    base_kwh_eur = 0.25
    min_kwh_eur = 0.21
    max_kwh_eur = 0.33

    parameter_values = [
    [min_factory_fix_costs, base_factory_fix_costs, max_factory_fix_costs],
    [min_factory_variable_costs, base_factory_variable_costs, max_factory_variable_costs],
    [min_factory_operating_costs, base_factory_operating_costs, max_factory_operating_costs],
    [min_qm_per_customer, base_qm_per_customer, max_qm_per_customer],
    [min_minimum_square_requirement, base_minimum_square_requirement, max_minimum_square_requirement],
    [min_rent_factor, base_rent_factor, max_rent_factor],
    [min_max_flight_distance, base_max_flight_distance, max_max_flight_distance],
    [min_drone_initial_costs, base_drone_initial_costs, max_drone_initial_costs],
    [min_drone_speed, base_drone_speed, max_drone_speed],
    [min_time_window, base_time_window, max_time_window],
    [min_delivery_time, base_delivery_time, max_delivery_time],
    [min_alpha, base_alpha, max_alpha],
    [min_night_shift_dist, base_night_shift_dist, max_night_shift_dist],
    [min_demand_factor, base_demand_factor, max_demand_factor],
    [min_watt_drone, base_watt_drone, max_watt_drone],
    [min_kwh_eur, base_kwh_eur, max_kwh_eur]
    ]
    return parameter_values

## Loops

### Optimaler Run

In [ ]:
optimal_run_results = []
folder = ['Wuerzburg_Data', 'Donner_Data', 'Frankfurt_Data']
city = ['wuerzburg', 'donner', 'frankfurt']

for i in range(3):

    customers_gdf = setup_customer_data(folder[i], city[i])
    geo_würzburg, bevölkerungs_gdf, pharmacy_gdf, warehouses_gdf, shifts_df = load_data(customers_gdf, folder[i], city[i])

    parameter_values = setup_parameters(shifts_df=shifts_df)

    optimal_run_results.append({
        'result_list': sensitivity_analysis(
                            parameter_values = parameter_values, 
                            number_of_runs = 1, 
                            sensitivity_index = 'None', 
                            parameter_names = parameter_names, 
                            optimal_run = True,
                            sensitivity_run = False,
                            bevölkerungs_gdf_run = bevölkerungs_gdf,
                            warehouses_gdf_run = warehouses_gdf,
                            customers_gdf_run = customers_gdf,
                            shifts_df_run = shifts_df,
                            city = city[i],
                            opened_warehouses_optimal = None,
                            warehouses_gdf_optimal = None,
                            customers_gdf_optimal = None,
                            bevölkerungs_gdf_optimal = None
                            ),
        'city': city[i]
    })

    with open(f'./Results/optimal_run_results.pkl', 'wb') as outfile:
        pickle.dump(optimal_run_results, outfile)

### Sensitivitätsanalyse

In [ ]:
def test_sensitivity(parameter_values, parameter_names, warehouses_gdf_run, customers_gdf_run, shifts_df_run, number_of_runs, sensitivity_index):
    
    triangular_parameter_values = [list(param) for param in parameter_values]

    if sensitivity_index != 'None':
        triangular_values = np.random.triangular(parameter_values[sensitivity_index][0], parameter_values[sensitivity_index][1], parameter_values[sensitivity_index][2], number_of_runs)
        triangular_values = np.append(triangular_values, parameter_values[sensitivity_index][0])
        triangular_values = np.append(triangular_values, parameter_values[sensitivity_index][2])

    result_list = []

    # Speichere den ursprünglichen Nachfragewert
    original_demand = customers_gdf_run['nachfrage'].copy()

    for i in range(len(triangular_values)):

        print(f'Triangular Value: {triangular_values[i]}')
    
        triangular_parameter_values[sensitivity_index][1] = triangular_values[i]

        base_values = [param[1] for param in triangular_parameter_values]

        base_dict = {
            'factory_fix_costs': base_values[0],
            'factory_variable_costs': base_values[1],
            'factory_operating_costs': base_values[2],
            'qm_per_customer': base_values[3],
            'minimum_square_requirement': base_values[4],
            'rent_factor': base_values[5],
            'max_flight_distance': base_values[6],
            'drone_initial_costs': base_values[7],
            'drone_speed': base_values[8],
            'time_window': base_values[9],
            'delivery_time': base_values[10],
            'alpha': base_values[11],
            'night_shift_dist': base_values[12],
            'demand_factor': base_values[13],
            'watt_drone': base_values[14],
            'kwh_eur': base_values[15]
        }

        # Setze die Nachfrage auf den ursprünglichen Wert zurück und multipliziere mit dem aktuellen demand_factor
        customers_gdf_run_demand = customers_gdf_run.copy()
        customers_gdf_run_demand['nachfrage'] = original_demand * base_dict['demand_factor']

        warehouses_gdf_filtered = warehouses_gdf_run[warehouses_gdf_run['floorSpace_small'] >= base_dict['minimum_square_requirement']].copy()
        
        # Drohnenparameter
        cost_per_km_drone = (base_dict['kwh_eur'] * base_dict['watt_drone']) / (base_dict['drone_speed'] * 60)  # EUR/km

        W = warehouses_gdf_filtered.index.values
        R = customers_gdf_run_demand.index.values
        S = shifts_df_run.index.values
        
        # Setze das Optimierungsproblem
        solver, x, y, z, d = optimize(
            W=W, 
            R=R,
            S=S, 
            warehouses_gdf_run=warehouses_gdf_filtered,
            customers_gdf_opt=customers_gdf_run_demand,
            cost_per_km_drone=cost_per_km_drone,
            factory_fix_costs=base_dict['factory_fix_costs'],
            factory_variable_costs=base_dict['factory_variable_costs'],
            factory_operating_costs=base_dict['factory_operating_costs'],
            qm_per_customer=base_dict['qm_per_customer'],
            rent_factor=base_dict['rent_factor'],
            max_flight_distance=base_dict['max_flight_distance'],
            drone_initial_costs=base_dict['drone_initial_costs'],
            drone_speed=base_dict['drone_speed'],
            time_window=base_dict['time_window'],
            alpha_drones=base_dict['alpha'],
            delivery_time=base_dict['delivery_time'],
            night_shift_dist=base_dict['night_shift_dist']
        )

        print('Solver set up!')

        # Löse das Problem und erhalte die Lösung
        opened_warehouses, customers_gdf_run, warehouses_gdf_filtered = solve(
            W=W,
            R=R, 
            solver=solver, 
            x=x, 
            y=y, 
            z=z, 
            d=d, 
            customers_gdf_run=customers_gdf_run_demand, 
            warehouses_gdf_run=warehouses_gdf_filtered
        )

        # Dictionary erstellen, um die Werte zu speichern
        warehouse_info = {}
        # Durch die geöffneten Lagerhäuser iterieren und die entsprechenden Werte hinzufügen
        for warehouse_id in opened_warehouses:
            if warehouse_id in warehouses_gdf_filtered.index:
                warehouse_info[warehouse_id] = {
                    'floor_space_assigned': warehouses_gdf_filtered.at[warehouse_id, 'floor_space_assigned'],
                    'number_of_drones': warehouses_gdf_filtered.at[warehouse_id, 'number_of_drones']
                }
                
        result_list.append({
            'parameter_name': parameter_names[sensitivity_index],
            'parameter_value': triangular_values[i],
            'opened_warehouses': np.array(opened_warehouses),
            'number_of_drones': np.sum(warehouses_gdf_filtered.loc[opened_warehouses].number_of_drones),
            'floor_space_assigned': np.sum(warehouses_gdf_filtered.loc[opened_warehouses].floor_space_assigned),
            'objective_value': solver.Objective().Value(),
            'Warehouse Info': warehouse_info,
            'parameter_values': triangular_parameter_values
        })

        print("-" * 50)
    
    return result_list


In [ ]:
sensitivity_run_results = []
folder = ['Wuerzburg_Data', 'Donner_Data', 'Frankfurt_Data']
city = ['wuerzburg', 'donner', 'frankfurt']

for i in range(3):

    customers_gdf = setup_customer_data(folder[i], city[i])
    geo_würzburg, bevölkerungs_gdf, pharmacy_gdf, warehouses_gdf, shifts_df = load_data(customers_gdf, folder[i], city[i])

    parameter_values = setup_parameters(shifts_df=shifts_df)
    print(f'City: {city[i]}')
    
    for j in range(len(parameter_names)):
        print(f'Parameter:{parameter_names[j]}')
        sensitivity_run_results.append({
            'result_list': test_sensitivity(
                                parameter_values=[list(param) for param in parameter_values],
                                number_of_runs = 50, 
                                sensitivity_index = j, 
                                parameter_names = parameter_names, 
                                warehouses_gdf_run = warehouses_gdf.copy(),
                                customers_gdf_run = customers_gdf.copy(),
                                shifts_df_run = shifts_df.copy()
                                ),
            'city': city[i]
        })

        with open('./Results/sensitivity_results.pkl', 'wb') as outfile:
            pickle.dump(sensitivity_run_results, outfile)

### Kostenvergleich der Edge Cases

In [ ]:
def comparison_simulate(opened_warehouses_run_sensitiv, 
                        bevölkerungs_gdf_run_sensitiv, 
                        warehouses_gdf_run_sensitiv,
                        opened_warehouses_run_optimal, 
                        bevölkerungs_gdf_run_optimal, 
                        warehouses_gdf_run_optimal,
                        cost_per_km_drone, 
                        drone_speed, 
                        delivery_time, 
                        start_time, 
                        end_time, 
                        demand_factor, 
                        rent_factor, 
                        night_shift_dist,
                        factory_fix_costs, 
                        factory_variable_costs, 
                        factory_operating_costs, 
                        drone_initial_costs):

    #Calculate the monthly fix costs due to drone and factory setup
    rental_cost_sensitiv = (np.sum(warehouses_gdf_run_sensitiv.loc[opened_warehouses_run_sensitiv].floor_space_assigned * warehouses_gdf_run_sensitiv.loc[opened_warehouses_run_sensitiv].pricePerSquareMetre)) * rent_factor
    factory_fix_monthly_sensitiv = (len(opened_warehouses_run_sensitiv) * factory_fix_costs) / 12
    factory_variable_monthly_sensitiv = np.sum(warehouses_gdf_run_sensitiv.loc[opened_warehouses_run_sensitiv].floor_space_assigned * factory_variable_costs) / 12
    factory_operating_monthly_sensitiv = np.sum(warehouses_gdf_run_sensitiv.loc[opened_warehouses_run_sensitiv].floor_space_assigned * warehouses_gdf_run_sensitiv.loc[opened_warehouses_run_sensitiv].pricePerSquareMetre * factory_operating_costs)
    drone_setup_cost_sensitiv = ((np.sum(warehouses_gdf_run_sensitiv.loc[opened_warehouses_run_sensitiv].number_of_drones) * drone_initial_costs) / 12) / 5 

    drone_transportation_cost_sensitiv = 0
    drone_transportation_time_sensitiv = 0
    median_penalty_time_sensitiv = 0
    median_waiting_time_sensitiv = 0
    avg_penalty_time_sensitiv = []
    avg_waiting_time_sensitiv = []


    #Calculate the monthly fix costs due to drone and factory setup
    rental_cost_optimal = (np.sum(warehouses_gdf_run_optimal.loc[opened_warehouses_run_optimal].floor_space_assigned * warehouses_gdf_run_optimal.loc[opened_warehouses_run_optimal].pricePerSquareMetre)) * rent_factor
    factory_fix_monthly_optimal = (len(opened_warehouses_run_optimal) * factory_fix_costs) / 12
    factory_variable_monthly_optimal = np.sum(warehouses_gdf_run_optimal.loc[opened_warehouses_run_optimal].floor_space_assigned * factory_variable_costs) / 12
    factory_operating_monthly_optimal = np.sum(warehouses_gdf_run_optimal.loc[opened_warehouses_run_optimal].floor_space_assigned * warehouses_gdf_run_optimal.loc[opened_warehouses_run_optimal].pricePerSquareMetre * factory_operating_costs)
    drone_setup_cost_optimal = ((np.sum(warehouses_gdf_run_optimal.loc[opened_warehouses_run_optimal].number_of_drones) * drone_initial_costs) / 12) / 5 

    drone_transportation_cost_optimal = 0
    drone_transportation_time_optimal = 0
    median_penalty_time_optimal = 0
    median_waiting_time_optimal = 0
    avg_penalty_time_optimal = []
    avg_waiting_time_optimal = []

    tqdm.pandas()

    #Loop der Simulation über ein gesamtes Jahr
    for i in range(30):

        print(f'Simulation - Tag: {i + 1}')
        
        #Sensitiv
        bevölkerungs_gdf_run_sensitiv['nachfrage'] = bevölkerungs_gdf_run_sensitiv.progress_apply(calculate_demand_sim, axis=1, demand_factor = demand_factor)
        bevölkerungs_gdf_run_sensitiv['demand_timestamp'] = bevölkerungs_gdf_run_sensitiv.progress_apply(lambda row: assign_random_timestamp(row, night_shift_dist, start_time, end_time), axis=1)

        # Calculate transportation costs and time using vectorized operations
        drone_transportation_cost_sensitiv += np.sum(bevölkerungs_gdf_run_sensitiv['nachfrage'] * bevölkerungs_gdf_run_sensitiv['distance_warehouse'] * cost_per_km_drone * 2)
        drone_transportation_time_sensitiv += np.sum((bevölkerungs_gdf_run_sensitiv['nachfrage'] * bevölkerungs_gdf_run_sensitiv['distance_warehouse']) / drone_speed )
        median_penalty_time_sensitiv, median_waiting_time_sensitiv  = calculate_penalty_costs(bevölkerungs_gdf_run_sensitiv, warehouses_gdf_run_sensitiv, drone_speed, delivery_time, start_time, end_time)
        avg_penalty_time_sensitiv.append(median_penalty_time_sensitiv)
        avg_waiting_time_sensitiv.append(median_waiting_time_sensitiv)
        
        bevölkerungs_gdf_run_sensitiv.drop('nachfrage', axis=1, inplace=True)
        bevölkerungs_gdf_run_sensitiv.drop('demand_timestamp', axis=1, inplace=True)


        #Optimal
        bevölkerungs_gdf_run_optimal['nachfrage'] = bevölkerungs_gdf_run_optimal.progress_apply(calculate_demand_sim, axis=1, demand_factor = demand_factor)
        bevölkerungs_gdf_run_optimal['demand_timestamp'] = bevölkerungs_gdf_run_optimal.progress_apply(lambda row: assign_random_timestamp(row, night_shift_dist, start_time, end_time), axis=1)

        # Calculate transportation costs and time using vectorized operations
        drone_transportation_cost_optimal += np.sum(bevölkerungs_gdf_run_optimal['nachfrage'] * bevölkerungs_gdf_run_optimal['distance_warehouse'] * cost_per_km_drone * 2)
        drone_transportation_time_optimal += np.sum((bevölkerungs_gdf_run_optimal['nachfrage'] * bevölkerungs_gdf_run_optimal['distance_warehouse']) / drone_speed )
        median_penalty_time_optimal, median_waiting_time_optimal  = calculate_penalty_costs(bevölkerungs_gdf_run_optimal, warehouses_gdf_run_optimal, drone_speed, delivery_time, start_time, end_time)
        avg_penalty_time_optimal.append(median_penalty_time_optimal)
        avg_waiting_time_optimal.append(median_waiting_time_optimal)

        bevölkerungs_gdf_run_optimal.drop('nachfrage', axis=1, inplace=True)
        bevölkerungs_gdf_run_optimal.drop('demand_timestamp', axis=1, inplace=True)



    return {
        'sensitiv':
        {
            'rental_cost': rental_cost_sensitiv,
            'factory_fix_monthly': factory_fix_monthly_sensitiv, 
            'factory_variable_monthly': factory_variable_monthly_sensitiv, 
            'factory_operating_monthly': factory_operating_monthly_sensitiv,
            'drone_setup_cost': drone_setup_cost_sensitiv, 
            'drone_transportation_cost': drone_transportation_cost_sensitiv * 15, 
            'drone_transportation_time': drone_transportation_time_sensitiv * 15, 
            'penalty_time': np.median(avg_penalty_time_sensitiv), 
            'waiting_time': np.median(avg_waiting_time_sensitiv)
        },
        'optimal':
        {
            'rental_cost': rental_cost_optimal,
            'factory_fix_monthly': factory_fix_monthly_optimal, 
            'factory_variable_monthly': factory_variable_monthly_optimal, 
            'factory_operating_monthly': factory_operating_monthly_optimal,
            'drone_setup_cost': drone_setup_cost_optimal, 
            'drone_transportation_cost': drone_transportation_cost_optimal * 15, 
            'drone_transportation_time': drone_transportation_time_optimal * 15, 
            'penalty_time': np.median(avg_penalty_time_optimal), 
            'waiting_time': np.median(avg_waiting_time_optimal)
        }
    }

In [ ]:
def setup_comparison_parameters():
    parameters = {
        'factory_fix_costs': 50000,
        'factory_variable_costs': 0.5,
        'factory_operating_costs': 0.375,
        'qm_per_customer': 0.5,
        'minimum_square_requirement': 750, # Quadratmeter
        'rent_factor': 1, # Prozent
        'max_flight_distance': 20, # km
        'drone_initial_costs': 4000, # Eur
        'drone_speed': 65 / 60, # km/h / 60 = km/min
        'time_window': 630, # Min
        'delivery_time': 60, # Min
        'alpha': 0.3,
        'night_shift_dist': 0.00112103746, # Prozent
        'demand_factor': 1, # Prozent
        'watt_drone': 1.08, # kW
        'kwh_eur': 0.25
    }
    return parameters

In [ ]:
folder = ['Wuerzburg_Data', 'Donner_Data', 'Frankfurt_Data']
city = ['wuerzburg', 'donner', 'frankfurt']
with open('./Results/optimal_run_results.pkl', 'rb') as infile:
    optimal_results = pickle.load(infile)

for i in range(1,3):

    print(f'City: {city[i]}')
    cost_comparison_result = []

    with open(f'./Results/parameter_comparison_{city[i]}.pkl', 'rb') as infile:
        parameter_change_list = pickle.load(infile)

    min_value = min(item['max_flight_distance'] for item in parameter_change_list if 'max_flight_distance' in item)
    parameter_change_list = [item for item in parameter_change_list if not ('max_flight_distance' in item and item['max_flight_distance'] == min_value)]


    for j in range(len(parameter_change_list)):

        key = 0
        value = 0

        for key, value in parameter_change_list[j].items():
            key = key
            value = value

        parameter_values = setup_comparison_parameters()
        parameter_values[key] = value

        print(f'Parameter: {key}, Wert: {value}')


        customers_gdf_optimal = gpd.read_file(f'./Results/Optimal_Results_customers_{city[i]}.gpkg')
        warehouses_gdf_optimal = gpd.read_file(f'./Results/Optimal_Results_warehouses_{city[i]}.gpkg')
        warehouses_gdf_optimal = warehouses_gdf_optimal.set_index('matching_index')
        bevölkerungs_gdf_optimal = gpd.read_file(f'./Results/Optimal_Results_bevoelkerung_{city[i]}.gpkg')
        bevölkerungs_gdf_optimal['Alter'] = bevölkerungs_gdf_optimal['Alter'].apply(json.loads)
        bevölkerungs_gdf_optimal['Geschlecht'] = bevölkerungs_gdf_optimal['Geschlecht'].apply(json.loads)
        opened_warehouses_optimal = optimal_results[i]['result_list'][0]['results']['opened_warehouses']


        customers_gdf_sensitiv = gpd.read_file(f'./Results/Optimal_Results_customers_{city[i]}.gpkg')
        customers_gdf_sensitiv.drop('assigned_warehouse', axis=1, inplace=True)
        geo_würzburg, bevölkerungs_gdf_sensitiv, pharmacy_gdf, warehouses_gdf_sensitiv, shifts_df = load_data(customers_gdf_sensitiv, folder[i], city[i])
        customers_gdf_sensitiv['nachfrage'] = customers_gdf_sensitiv['nachfrage']  * parameter_values['demand_factor']
        warehouses_gdf_sensitiv = warehouses_gdf_sensitiv[warehouses_gdf_sensitiv['floorSpace_small'] >= parameter_values['minimum_square_requirement']]

        parameter_values['time_window'] = int(parameter_values['time_window'])
        parameter_values['delivery_time'] = int(parameter_values['delivery_time'])
        start_time, end_time = calculate_times(minutes=parameter_values['time_window'])
        cost_per_km_drone = (parameter_values['kwh_eur'] * parameter_values['watt_drone']) / (parameter_values['drone_speed'] * 60)  # EUR/km # Calculate cost per kilometer for the drone

        W = warehouses_gdf_sensitiv.index.values
        R = customers_gdf_sensitiv.index.values
        S = shifts_df.index.values
        
        # Set the optimization problem
        solver, x, y, z, d = optimize(
            W = W, 
            R = R,
            S = S, 
            warehouses_gdf_run = warehouses_gdf_sensitiv,
            customers_gdf_opt = customers_gdf_sensitiv, 
            cost_per_km_drone = cost_per_km_drone,
            factory_fix_costs = parameter_values['factory_fix_costs'],
            factory_variable_costs = parameter_values['factory_variable_costs'],
            factory_operating_costs =  parameter_values['factory_operating_costs'],
            qm_per_customer = parameter_values['qm_per_customer'],
            rent_factor = parameter_values['rent_factor'],
            max_flight_distance = parameter_values['max_flight_distance'],
            drone_initial_costs = parameter_values['drone_initial_costs'],
            drone_speed = parameter_values['drone_speed'],
            time_window = parameter_values['time_window'],
            alpha_drones = parameter_values['alpha'],
            delivery_time = parameter_values['delivery_time'],
            night_shift_dist = parameter_values['night_shift_dist']
            )
        
        # Löse das Problem und erhalte die Lösung
        opened_warehouses_sensitiv, customers_gdf_sensitiv, warehouses_gdf_sensitiv = solve(
            W=W,
            R=R, 
            solver=solver, 
            x=x, 
            y=y, 
            z=z, 
            d=d, 
            customers_gdf_run=customers_gdf_sensitiv, 
            warehouses_gdf_run=warehouses_gdf_sensitiv
        )

        bevölkerungs_gdf_sensitiv = assign_warehouses(bevölkerungs_gdf_run=bevölkerungs_gdf_sensitiv, 
                                                      customers_gdf_run=customers_gdf_sensitiv,
                                                      warehouses_gdf_run=warehouses_gdf_sensitiv)

        compared_costs = comparison_simulate(opened_warehouses_run_sensitiv=opened_warehouses_sensitiv,
                                             bevölkerungs_gdf_run_sensitiv=bevölkerungs_gdf_sensitiv,
                                             warehouses_gdf_run_sensitiv=warehouses_gdf_sensitiv,
                                             opened_warehouses_run_optimal=opened_warehouses_optimal,
                                             bevölkerungs_gdf_run_optimal=bevölkerungs_gdf_optimal,
                                             warehouses_gdf_run_optimal=warehouses_gdf_optimal,
                                             cost_per_km_drone=cost_per_km_drone,
                                             drone_speed=parameter_values['drone_speed'],
                                             delivery_time=parameter_values['delivery_time'],
                                             start_time=start_time,
                                             end_time=end_time,
                                             demand_factor=parameter_values['demand_factor'],
                                             rent_factor=parameter_values['rent_factor'],
                                             night_shift_dist=parameter_values['night_shift_dist'],
                                             factory_fix_costs=parameter_values['factory_fix_costs'],
                                             factory_variable_costs=parameter_values['factory_variable_costs'],
                                             factory_operating_costs=parameter_values['factory_operating_costs'],
                                             drone_initial_costs=parameter_values['drone_initial_costs'])

        cost_comparison_result.append({
            'parameter': key,
            'parameter_value': value,
            'compared_costs': compared_costs,
        })
        
        with open(f'./Results/cost_comparison_{city[i]}.pkl', 'wb') as outfile:
            pickle.dump(cost_comparison_result, outfile)

# Tests

## testing

In [ ]:
sensitivity_run_results = []
folder = ['Wuerzburg_Data', 'Donner_Data', 'Frankfurt_Data']
city = ['wuerzburg', 'donner', 'frankfurt']

for i in range(3):

    customers_gdf = setup_customer_data(folder[i], city[i])
    geo_würzburg, bevölkerungs_gdf, pharmacy_gdf, warehouses_gdf, shifts_df = load_data(customers_gdf, folder[i], city[i])
    
    with open('./Results/optimal_run_results.pkl', 'rb') as infile:
        optimal_result = pickle.load(infile)
        opened_warehouses = optimal_result[i]['result_list'][0]['results']['opened_warehouses']

    parameter_values = setup_parameters(shifts_df=shifts_df)

    customers_gdf_optimal = gpd.read_file(f'./Results/Optimal_Results_customers_{city[i]}.gpkg')
    customers_gdf_optimal['Alter'] = customers_gdf_optimal['Alter'].apply(json.loads)

    warehouses_gdf_optimal = gpd.read_file(f'./Results/Optimal_Results_warehouses_{city[i]}.gpkg')

    bevölkerungs_gdf_optimal = gpd.read_file(f'./Results/Optimal_Results_bevoelkerung_{city[i]}.gpkg')
    bevölkerungs_gdf_optimal['Alter'] = bevölkerungs_gdf_optimal['Alter'].apply(json.loads)
    bevölkerungs_gdf_optimal['Geschlecht'] = bevölkerungs_gdf_optimal['Geschlecht'].apply(json.loads)
    
    for i in range(len(parameter_values)):    
        sensitivity_run_results.append({
            'result_list': sensitivity_analysis(
                                parameter_values = parameter_values, 
                                number_of_runs = 10, 
                                sensitivity_index = i, 
                                parameter_names = parameter_names, 
                                optimal_run = False,
                                sensitivity_run = True,
                                bevölkerungs_gdf_run = bevölkerungs_gdf,
                                warehouses_gdf_run = warehouses_gdf,
                                customers_gdf_run = customers_gdf,
                                shifts_df_run = shifts_df,
                                city = city[i],
                                opened_warehouses_optimal = opened_warehouses,
                                customers_gdf_optimal = customers_gdf_optimal,
                                warehouses_gdf_optimal = warehouses_gdf_optimal, 
                                bevölkerungs_gdf_optimal = bevölkerungs_gdf_optimal  
                                ),
            'city': city[i]
        })

        with open('./Results/sensitivity_results.pkl', 'wb') as outfile:
            pickle.dump(sensitivity_run_results, outfile)

## datafix

## energy_costs

In [ ]:
res = requests.get('https://api.corrently.io/v2.0/gsi/marketdata?zip=97072')

In [ ]:
content = json.loads(res.content)

In [ ]:
content

In [ ]:
# Assuming the response data is stored in a variable called 'response_data'

# Extract market prices
market_prices = [item['localprice'] for item in content['data']]

# Sort the market prices
sorted_prices = sorted(market_prices)

# Calculate the number of data points
num_prices = len(sorted_prices)

# Calculate median position
median_pos = (num_prices + 1) // 2

# Access the median value
median = sorted_prices[median_pos - 1] / 1000

# Calculate quartile positions (rounded down)
q1_pos = (num_prices + 1) // 4
q3_pos = 3 * (num_prices + 1) // 4

# Access the quartile values
q1 = sorted_prices[q1_pos - 1] / 1000
q3 = sorted_prices[q3_pos - 1] / 1000

# Print the results
print("Median market price:", median, "Eur/MWh")
print("25th percentile (Q1):", q1, "Eur/MWh")
print("75th percentile (Q3):", q3, "Eur/MWh")


In [ ]:
gdf_1 = gpd.read_file('./Results/Optimal_Results_warehouses_wuerzburg.gpkg')
gdf_2 = gpd.read_file('./Wuerzburg_Data/warehouses_wuerzburg.gpkg')

In [ ]:
gdf_1

In [ ]:
for idx1, row1 in gdf_1.iterrows():
    matching_idx = gdf_2[gdf_2['title'] == row1['title']].index
    if not matching_idx.empty:
        gdf_1.at[idx1, 'matching_index'] = int(matching_idx[0])

In [ ]:
gdf_1.to_file('./Results/Optimal_Results_warehouses_wuerzburg.gpkg', driver='GPKG')

# Sim Tests

In [ ]:
with open('./Results/optimal_run_results.pkl', 'rb') as infile:
    solution2 = pickle.load(infile)

In [ ]:
# Keys to remove
keys_to_remove = {'factory_fix_costs', 'qm_per_customer', 'qm_per_drone', 'max_flight_distance', 'drone_initial_costs'}

# Filter out dictionaries that contain any of the keys
filtered_solution = [entry for entry in solution if not any(key in entry for key in keys_to_remove)]

filtered_solution

In [ ]:
with open('./Results/parameter_comparison_filtered_1.pkl', 'wb') as outfile:
    pickle.dump(filtered_solution, outfile)

In [ ]:
sensitivity_run_results = []
folder = ['Wuerzburg_Data', 'Donner_Data', 'Frankfurt_Data']
city = ['wuerzburg', 'donner', 'frankfurt']

for i in range(2,3):

    customers_gdf2 = setup_customer_data(folder[i], city[i])

In [ ]:
customers_gdf2.to_file('./Frankfurt_Data/cluster_frankfurt.gpkg', driver='GPKG')

In [55]:
with open('./Results/optimal_run_results.pkl', 'rb') as infile:
    solution = pickle.load(infile)

solution

[{'result_list': [{'parameter': 'Base Value',
    'parameter_value': 'Base_Value',
    'results': {'opened_warehouses': array([ 2, 24, 35]),
     'number_of_drones': 418,
     'floor_space_assigned': 10743,
     'objective_value': 1236137.4667776353,
     'Factory Rental Cost': 53303.6,
     'Factory Fix Cost': 12500.0,
     'Factory Variable Cost': 447.625,
     'Factory Operating Cost': 19988.85,
     'Drone Setup Cost': 27866.666666666668,
     'Drone Transportation Cost': 12880.076253767085,
     'Drone Transportation Time': 23851.99306253165,
     'Transportation Penalty Cost': 219.95,
     'Transportation Avg Waiting Time': 11.5,
     'Car/Truck Customer Transportation Cost': 0,
     'Car/Truck Customer Transportation Time': 0,
     'Car/Truck Pharmacy Transportation Cost': 0,
     'Car/Truck Pharmacy Transportation Time': 0,
     'Warehouse Info': {2: {'floor_space_assigned': 2950,
       'number_of_drones': 275},
      24: {'floor_space_assigned': 6993, 'number_of_drones': 78},